# Announcement

I made a big switch here. In video I use the Pima dataset from Midterm 1. But I decided it was too confusing to start messing with this dataset during the final project. It seemed a lot more consistent to stick with Titanic, so that is what I did. Steps are the same, with caveat below.

I made a change to  *Feature reduction*. I have given you a new way (new library) that is more useful than the pandas `.corr` method.


I also removed the need to save the test set. We really do not need it.

Finally, I am asking you to build the markdown (md) desription of your pipeline including a screenshot of it. Refer back end to Chapter 7 for a model of what you need. You may be able to copy and paste what I have in Chapter 7 then edit it to fit your new pipeline.

<center>
<h1>First Notebook for Final Project (Wrangling)</h1>
</center>

<hr>

I am using Titanic dataset. You will need to fill in with your own. So replace my csv, random_state and pipeline with your own.

Reminder: here is a list of places you can find data to use: https://careerfoundry.com/en/blog/data-analytics/where-to-find-free-datasets/.

# Notes on your choice of dataset

* I would like you to find a csv dataset that has a mixture of numeric and categorical columns. So you can show off your transformers!

* I'd like a binary label column, i.e., what you will predict.

* Do not worry about number of rows. If they are below 500 you can use SMOTE to increase them. If they are over 5000, you can use downsample to decrease them.

* Don't worry about number of feature columns. I can help you reduce a large number down to 10 or less.


# I'd like to know the dataset you choose. Email me a link.

Do this before starting this notebook. I may have some guidance that will help you and potentially direct you off of dead-end paths.

## Set-up

First bring in your library.

In [1]:
github_name = 'marvnc'
repo_name = 'cs523'
source_file = 'library.py'
url = f'https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}'
!rm $source_file
!wget $url
%run -i $source_file
# from library import *

rm: cannot remove 'library.py': No such file or directory
--2025-06-06 20:59:08--  https://raw.githubusercontent.com/marvnc/cs523/main/library.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49866 (49K) [text/plain]
Saving to: ‘library.py’

library.py          100%[===================>]  48.70K  --.-KB/s    in 0.01s   

2025-06-06 20:59:08 (3.80 MB/s) - ‘library.py’ saved [49866/49866]



## Demo with Titanic

## Caveat

You will not match my results. You have your own dataset. I am just giving you the outline of the steps I expect you to follow.

But you will need your dataset stored in a way that you can load it (and I can load it). This may mean downloading from web (e.g., kaggle) and then uploading to GitHub.

In [2]:
# url = 'https://raw.githubusercontent.com/fickas/asynch_models/main/datasets/titanic_trimmed.csv'  #trimmed version
# path = 's25_final_cc_approvals.csv'
path = 'https://raw.githubusercontent.com/MarvNC/cs523/main/s25_final_cc_approvals.csv'

cc_approvals = pd.read_csv(path)
cc_approvals.head()


,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,Industrials,White,1.25,1,1,1,0,ByBirth,202,0,1
1,0,58.67,4.460,1,1,Materials,Black,3.04,1,1,6,0,ByBirth,43,560,1
2,0,24.50,0.500,1,1,Materials,Black,1.50,1,0,0,0,ByBirth,280,824,1
3,1,27.83,1.540,1,1,Industrials,White,3.75,1,1,5,1,ByBirth,100,3,1
4,1,20.17,5.625,1,1,Industrials,White,1.71,1,0,0,0,ByOtherMeans,120,0,1


In [3]:
len(cc_approvals)

690

# Break out into features and labels



In [4]:
feature_col = 'Approved'
approval_features = cc_approvals.drop(columns=feature_col)
labels = cc_approvals[feature_col].to_list()

In [5]:
labels.count(1)/len(labels)
# We see that 44% of the rows were approved here

0.4449275362318841

# Downsampling (optional)

Here is code to reduce the number of rows while keeping the new table stratified on the target column.

You can skip this step if your table has less than 5000 rows. You could probably skip it with less than 10000. The whole point is to avoid tuning (next notebook) that takes hours. In a real setting, you would probably have to bite the bullet and wait that long.

Note that I am setting N=1000 just for demo purposes. You should use 5000 or even greater as your value if you do need to downsample.

In [6]:
# N=1000  #size of table you want, really not needed for such a small table as Titanic
# target = feature_col
# original_df = cc_approvals.copy()

# downsample_df = original_df.groupby(target, group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(original_df))))).sample(frac=1).reset_index(drop=True)
# downsample_df

# We don't need to downsample since we only have 690 rows.

In [7]:
# downsample_df['Survived'].to_list().count(1)/len(downsample_df)  #same as original so seems to work

# Also unneeded

### For those who want to know

Here is brief explanation:

* `original_df.groupby(target, group_keys=False)`: This groups the dataframe by the target column. The `group_keys=False` ensures that the original grouping columns are not included in the resulting dataframe.

* The apply method is used to apply a function to each group:

 * `lambda x: x.sample(int(np.rint(N*len(x)/len(original_df))))`: This function samples from each group based on the ratio of the group size to the total dataframe size multiplied by N (the number of samples you want from the entire dataframe).
 * The `np.rint` function rounds the number to the nearest integer.
 * `sample(frac=1)`: This shuffles the rows of the resulting dataframe.

* reset_index(drop=True): This resets the index of the dataframe and drops the original index.

In essence, the code calculates the number of samples needed from each group directly, based on the proportion of the group in the original dataframe, and then samples that number of rows from each group. It then shuffles the combined downsampled dataframe.

# Feature reduction (optional)

Some will need to drop columns. I'll give you a new library.  I like it because it works with a table with a mixture of datatypes, e.g., categorical columns.

You should be able to use `correlation_df` (below) in place of the code you worked on earlier that used the pandas correlation table.

You can play around with the threshold to get to your target number of features.



### New correlation method

The `titanic_trimmed.corr()` given  will give you an error if you have mixed column types. This new method handles this fine. When comparing two columns, it uses:

* Cramer's V for categorical-categorical.
* correlation ratio for numeric-categorical.
* Pearson for numeric-numeric.

In [8]:
# !pip install dython


In [13]:
# from dython.nominal import associations

categorical_vars = [
    "Gender",
    "Married",
    "BankCustomer",
    "Industry",
    "Ethnicity",
    "PriorDefault",
    "Employed",
    "DriversLicense",
    "Citizen",
    "ZipCode",
]

# # Simple one-liner to get correlations:
# correlations = associations(
#     cc_approvals, nominal_columns=categorical_vars
# )  # you have to tell it which are categorical
# correlations

In [11]:
# # Convert to dataframe
# correlation_df = correlations['corr']
# correlation_df

In [14]:
#Rank feature importance - ignore warnings

import lightgbm as lgb

temp_features_df = approval_features.copy()

categorical_features = categorical_vars

# Convert categorical columns to 'category' dtype
for col in categorical_features:
 temp_features_df[col] = temp_features_df[col].astype('category')

# Initialize and train the LightGBM model, specifying categorical features

lgbm = lgb.LGBMClassifier(categorical_feature=categorical_features)
lgbm.fit(temp_features_df, labels)

# Get feature importances
feature_importances = lgbm.feature_importances_

# Create a pandas Series for better visualization
feature_importance_series = pd.Series(feature_importances, index=temp_features_df.columns)

# Sort the feature importances in descending order
sorted_feature_importance = feature_importance_series.sort_values(ascending=False)

# Print the ranked feature importance
print("Feature Importance Ranking (LightGBM):")
print(sorted_feature_importance)

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[LightGBM] [Warning] categorical_feature is set=Gender,Married,BankCustomer,Industry,Ethnicity,PriorDefault,Employed,DriversLicense,Citizen,ZipCode, categorical_column=0,3,4,5,6,8,9,11,12,13 will be ignored. Current value: categorical_feature=Gender,Married,BankCustomer,Industry,Ethnicity,PriorDefault,Employed,DriversLicense,Citizen,ZipCode
[LightGBM] [Info] Number of positive: 307, number of negative: 383
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000347 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 625
[LightGBM] [Info] Number of data points in the train set: 690, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444928 -> initscore=-0.221187
[LightGBM] [Info] Start training from score -0.221187
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

## If you downsample and/or do feature reduction

Save the resulting table out to github so you do not have to keep repeating the process.


In [ ]:
# Remove less important features (arbitrarily remove stuff with ranking under 50 from lightBGM)

less_important = [
    'Married',
    'Industry',
    'ZipCode',
    'Citizen',
    'BankCustomer',
    'Ethnicity'
]

# Drop
reduced_df = cc_approvals.drop(columns=less_important)

# Save to csv
reduced_df.to_csv('s25_final_cc_approvals_reduced.csv', index=False)

# Define your pipeline

This is up to you for your own dataset.


In [16]:
# Show remaining columns
reduced_df.columns

Index(['Gender', 'Age', 'Debt', 'YearsEmployed', 'PriorDefault', 'Employed',
       'CreditScore', 'DriversLicense', 'Income', 'Approved'],
      dtype='object')

In [17]:

approvals_transformer = Pipeline(steps=[
    # Gender: already categorical 0 or 1
    # Age: numerical, so we might transform to normalize it then apply tukey for outliers
    ('tukey_age', CustomTukeyTransformer(target_column='Age', fence='outer')),
    ('scale_age', CustomRobustTransformer(target_column='Age')),
    # Debt: numerical, so normalize and apply tukey
    ('tukey_debt', CustomTukeyTransformer(target_column='Debt', fence='outer')),
    ('scale_debt', CustomRobustTransformer(target_column='Debt')),
    # YearsEmployed numerical
    ('tukey_years_employed', CustomTukeyTransformer(target_column='YearsEmployed', fence='outer')),
    ('scale_years_employed', CustomRobustTransformer(target_column='YearsEmployed')),
    # PriorDefault is already categorical 0 or 1
    # Employed is already categorical 0 or 1
    # CreditScore is numerical
    ('tukey_credit_score', CustomTukeyTransformer(target_column='CreditScore', fence='outer')),
    ('scale_credit_score', CustomRobustTransformer(target_column='CreditScore')),
    # DriversLicense is already categorical 0 or 1
    # Income is numerical
    ('tukey_income', CustomTukeyTransformer(target_column='Income', fence='outer')),
    ('scale_income', CustomRobustTransformer(target_column='Income')),

    # Impute missing values
    ("impute", CustomKNNTransformer(n_neighbors=5)),
    ], verbose=True)


## Take a screenshot of your pipeline

Save it on GitHub. Then use Chapter 7 markdown example to build your own pipeline description. Also save this on GitHub.

## Find random state for splitting



In [18]:
approval_features = reduced_df.drop(columns=feature_col)
labels = reduced_df[feature_col].to_list()

In [19]:
approvals_transformed_df = approvals_transformer.fit_transform(approval_features, labels)

[Pipeline] ........ (step 1 of 11) Processing tukey_age, total=   0.0s
[Pipeline] ........ (step 2 of 11) Processing scale_age, total=   0.0s
[Pipeline] ....... (step 3 of 11) Processing tukey_debt, total=   0.0s
[Pipeline] ....... (step 4 of 11) Processing scale_debt, total=   0.0s
[Pipeline]  (step 5 of 11) Processing tukey_years_employed, total=   0.0s
[Pipeline]  (step 6 of 11) Processing scale_years_employed, total=   0.0s
[Pipeline]  (step 7 of 11) Processing tukey_credit_score, total=   0.0s
[Pipeline]  (step 8 of 11) Processing scale_credit_score, total=   0.0s
[Pipeline] ..... (step 9 of 11) Processing tukey_income, total=   0.0s
[Pipeline] .... (step 10 of 11) Processing scale_income, total=   0.0s
[Pipeline] .......... (step 11 of 11) Processing impute, total=   0.0s


In [20]:
approvals_transformed_df.head()

,Gender,Age,Debt,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Income
0,1.0,0.157606,-0.443012,0.101626,1.0,1.0,0.333333,0.0,-0.012642
1,0.0,2.008978,0.275473,0.829268,1.0,1.0,2.000000,0.0,1.403287
2,0.0,-0.263342,-0.362465,0.203252,1.0,0.0,0.000000,0.0,2.070796
3,1.0,-0.041895,-0.194925,1.117886,1.0,1.0,1.666667,1.0,-0.005057
4,1.0,-0.551288,0.463149,0.288618,1.0,0.0,0.000000,0.0,-0.012642


# Find random_state value



In [21]:
%%capture
rs, _ = find_random_state(approvals_transformed_df, labels, approvals_transformer)

In [22]:
rs  #whatever it is for your own dataset
# We got 174

np.int64(174)

### Remember this value for next notebook, tuning

## Split your dataset and save fitted pipeline

In [23]:
X_train, X_test, y_train, y_test = train_test_split(approval_features, labels, test_size=0.2, random_state=rs)
production_pipeline = approvals_transformer.fit(X_train, y_train)  #now ready for production

[Pipeline] ........ (step 1 of 11) Processing tukey_age, total=   0.0s
[Pipeline] ........ (step 2 of 11) Processing scale_age, total=   0.0s
[Pipeline] ....... (step 3 of 11) Processing tukey_debt, total=   0.0s
[Pipeline] ....... (step 4 of 11) Processing scale_debt, total=   0.0s
[Pipeline]  (step 5 of 11) Processing tukey_years_employed, total=   0.0s
[Pipeline]  (step 6 of 11) Processing scale_years_employed, total=   0.0s
[Pipeline]  (step 7 of 11) Processing tukey_credit_score, total=   0.0s
[Pipeline]  (step 8 of 11) Processing scale_credit_score, total=   0.0s
[Pipeline] ..... (step 9 of 11) Processing tukey_income, total=   0.0s
[Pipeline] .... (step 10 of 11) Processing scale_income, total=   0.0s
[Pipeline] .......... (step 11 of 11) Processing impute, total=   0.0s


In [24]:
import joblib

joblib.dump(production_pipeline, "final_fully_fitted_pipeline.pkl")  #Move this to GitHub where you will use it in production

['final_fully_fitted_pipeline.pkl']

In [25]:
ptransformer = joblib.load("final_fully_fitted_pipeline.pkl")  # Make sure can get it back


# Congrats

You are done with the wrangling stage. Ready to move on to training and tuning models.

# Reminder

You should have the following files stored on GitHub.

1. Screenshot of pipeline.

2. Markdown pipeline description that references screenshot.

3. `final_fully_fitted_pipeline.pkl`

4. Value for `rs` to use in train_test_split. Saving this to GitHub is optional. You can just remember it and use it in tuning notebook.